In [45]:
#import requirements
import google.generativeai as genai
import json
import time
from google.api_core.exceptions import TooManyRequests

# set your API key
from google.colab import userdata
userdata.get('google_key')

#define model
model = genai.GenerativeModel("gemini-2.5-flash")

# Agent Training

In [34]:
#add in xyz memory
MEMORY_FILE = "memory.json"
def load_memory():
    try:
        with open(MEMORY_FILE, "r") as f:
            return json.load(f)
    except:
        return []

def save_memory(chat):
    simple_history = []
    for msg in chat.history:
        role = msg.role
        text = "".join(part.text for part in msg.parts)
        simple_history.append({"role": role, "text": text})

    with open(MEMORY_FILE, "w") as f:
        json.dump(simple_history, f, indent=2)

history = load_memory()

# convert stored history back to Gemini format
formatted_history = [
    {"role": h["role"], "parts": [h["text"]]} for h in history
]

chat = model.start_chat(history=formatted_history)

print("Chat with file memory (type exit)")

while True:
    user_input = input("\nYou: ")
    if user_input.lower() == "exit":
        break

    try:
      response = chat.send_message(user_input)
      print("Bot:", response.text)
    except TooManyRequests as e:
      wait = int(e.retry_delay.seconds) if hasattr(e, "retry_delay") else 40
      print(f"Rate limit hit. Waiting {wait}s...")
      time.sleep(wait)
      print("Bot:", response.text)

    save_memory(chat)

Chat with file memory (type exit)

You: exit


# Aizaz's Resume Agent

In [31]:
#pip install google-generativeai PyPDF2 python-docx

In [46]:
import google.generativeai as genai
import PyPDF2
from docx import Document

In [47]:
R_MEMORY_FILE = "resume_memory.json"
MAX_TURNS = 5


In [48]:
def load_pdf(file_path):
    text = ""
    with open(file_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() + "\n"
    return text

In [49]:
def load_docx(file_path):
    doc = Document(file_path)
    text = "\n".join([p.text for p in doc.paragraphs])
    return text

In [51]:
def load_memory():
    try:
        with open(R_MEMORY_FILE, "r") as f:
            return json.load(f)
    except:
        return []

In [52]:
def save_memory(chat):
    simple = []
    for msg in chat.history[-MAX_TURNS:]:
        text = "".join(p.text for p in msg.parts)
        simple.append({"role": msg.role, "text": text})

    with open(R_MEMORY_FILE, "w") as f:
        json.dump(simple, f, indent=2)

In [53]:
def format_history_for_chat(history):
    return [{"role": h["role"], "parts": [h["text"]]} for h in history]


In [54]:
import re
def extract_details(text):
    details = {}

    # Name: assume first line with letters only
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    details['Name'] = lines[0] if lines else "N/A"

    # Email
    email_match = re.search(r'\b[\w.-]+@[\w.-]+\.\w+\b', text)
    details['Email'] = email_match.group(0) if email_match else "N/A"

    # Phone
    phone_match = re.search(r'\+?\d[\d\s.-]{7,}\d', text)
    details['Phone'] = phone_match.group(0) if phone_match else "N/A"

    # Skills (simple example: look for lines containing 'Skills' or bullet points)
    skills_lines = [line for line in lines if "skill" in line.lower()]
    details['Skills'] = ", ".join(skills_lines) if skills_lines else "N/A"

    # Education (look for common degrees)
    edu_lines = [line for line in lines if any(deg in line.lower() for deg in ['bachelor','master','phd','msc','bsc'])]
    details['Education'] = ", ".join(edu_lines) if edu_lines else "N/A"

    # Experience (look for 'Experience', 'Internship', 'Worked at')
    exp_lines = [line for line in lines if any(word in line.lower() for word in ['experience','intern','worked at'])]
    details['Experience'] = ", ".join(exp_lines) if exp_lines else "N/A"

    return details

In [55]:
def save_details(details, filename="resume_details.txt"):
    with open(filename, "w") as f:
        for k, v in details.items():
            f.write(f"{k}: {v}\n")
    print(f"Details saved to {filename}")

In [56]:
from os import name
resume_text = load_pdf("/content/drive/MyDrive/Personal Documents/Aizaz-Ali-Shah-CV2025.pdf")   # or load_docx("resume.docx")
details = extract_details(resume_text)
save_details(details)
history = load_memory()
formatted_history = format_history_for_chat(history)
chat = model.start_chat(history=formatted_history)

print("Resume Chatbot started! Type 'exit' to quit.\n")
name = "Aizaz Ali Shah"
summary=f"I am a data science graduate with experience in speech technology, NLP, and machine learning. Professionally, I manage projects, ensuring quality and efficiency across software and research initiatives. I have a strong background in ASR (automatic speech recognition) projects, including research publications and patents, and I enjoy applying AI to practical problems. Beyond tech, I have experience in teaching, project management, and customer-focused solutions, blending technical expertise with clear communication and strategic thinking."
while True:
    question = input("You: ")
    if question.lower() == "exit":
        break

    # prepend resume context to question
    system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

    system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{resume_text[:1000]}\n\n"
    system_prompt += f"With this context, please chat with the user, always staying in character as {name}."
    try:
        response = chat.send_message(system_prompt)
        print("Bot:", response.text)
        break
    except TooManyRequests:
        print("Rate limit hit. Waiting 30s...")
        time.sleep(30)
    save_memory(chat)

Details saved to resume_details.txt
Resume Chatbot started! Type 'exit' to quit.

You: who are you?


Rate limit hit. Waiting 30s...
You: exit


In [57]:
!git init

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [58]:
!git add .
!git commit -m "Initial commit: smart resume parser"


error: open("drive/MyDrive/1-TH-3 Form  4 x copies.gdoc"): Operation not supported
error: unable to index file 'drive/MyDrive/1-TH-3 Form  4 x copies.gdoc'
fatal: adding files failed
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@5f1207b56a99.(none)')
